# **Introduction**

This notebook serves as an implementation of Soft Actor-Critic (SAC) on the custom-developed 2D navigation environment, titled ``Nav2D-v0``. The goal of this implementation is to quantify the performance of SAC in a simple 2D navigational  task, such that it can be used for incremental learning within subsequent environments.

# **Imports**

This section imports the necessary packages for this implementation.

In [9]:
# import gymnasium related packages:
import gymnasium as gym
from gymnasium.utils.env_checker import check_env
from gymnasium.wrappers import TimeLimit

# import the custom environment:
import nav2d


check env:

In [12]:
# make the environment:
env = TimeLimit(gym.make("Nav2D-v0"), max_episode_steps = 500)

# check the environment:
try: 
    check_env(env.unwrapped)
    print(f"Environment passes all checks!")
except Exception as e:
    print(f"Environment has the following issues: \n{e}")

Environment has the following issues: 
Expects the random number generator to have been generated given a seed was passed to reset. Most likely the environment reset function does not call `super().reset(seed=seed)`.


c:\Users\mtidd\OneDrive\Desktop\drl_navigation\.venv\Lib\site-packages\gymnasium\utils\env_checker.py:333: UserWarning: WARN: For Box action spaces, we recommend using a symmetric and normalized space (range=[-1, 1] or [0, 1]). See https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html for more information.
  logger.warn(
c:\Users\mtidd\OneDrive\Desktop\drl_navigation\.venv\Lib\site-packages\gymnasium\utils\env_checker.py:311: UserWarning: WARN: A Box observation space minimum value is -infinity. This is probably too low.
  logger.warn(
c:\Users\mtidd\OneDrive\Desktop\drl_navigation\.venv\Lib\site-packages\gymnasium\utils\env_checker.py:315: UserWarning: WARN: A Box observation space maximum value is infinity. This is probably too high.
  logger.warn(
